In [8]:
import pandas as pd
import os
from natsort import natsorted
import glob
import re
from pathlib import Path

In [ ]:
#Make list of files

files = glob.glob("crispresso_outputs_JAK2"+"/*/")
print(files)

MIN_READS = 500
CO_MUT = 0.125
CO_HOM = 0.15
CO_HET = 0.8


['crispresso_outputs_JAK2/CRISPResso_on_S051_S51_L001_R1_001_P06_S051_S51_L001_R2_001_P06/', 'crispresso_outputs_JAK2/CRISPResso_on_S204_S204_L001_R1_001_P02_S204_S204_L001_R2_001_P02/', 'crispresso_outputs_JAK2/CRISPResso_on_S285_S285_L001_R1_001_P04_S285_S285_L001_R2_001_P04/', 'crispresso_outputs_JAK2/CRISPResso_on_S071_S71_L001_R1_001_P04_S071_S71_L001_R2_001_P04/', 'crispresso_outputs_JAK2/CRISPResso_on_S026_S26_L001_R1_001_P04_S026_S26_L001_R2_001_P04/', 'crispresso_outputs_JAK2/CRISPResso_on_S111_S111_L001_R1_001_P03_S111_S111_L001_R2_001_P03/', 'crispresso_outputs_JAK2/CRISPResso_on_S026_S26_L001_R1_001_P03_S026_S26_L001_R2_001_P03/', 'crispresso_outputs_JAK2/CRISPResso_on_S212_S212_L001_R1_001_P04_S212_S212_L001_R2_001_P04/', 'crispresso_outputs_JAK2/CRISPResso_on_S123_S123_L001_R1_001_P03_S123_S123_L001_R2_001_P03/', 'crispresso_outputs_JAK2/CRISPResso_on_S372_S372_L001_R1_001_P04_S372_S372_L001_R2_001_P04/', 'crispresso_outputs_JAK2/CRISPResso_on_S357_S357_L001_R1_001_P02_S3

In [ ]:
data = []


for file in natsorted(files):
    cell = {}
    path_indels = file + '/CRISPResso_quantification_of_editing_frequency.txt'
    if Path(path_indels).exists():
        df_percentage = df = pd.read_csv(path_indels, delimiter='\t')
        cell['reads_WT'] = df_percentage.iloc[0,5]
        cell['reads_mutant'] = df_percentage.iloc[1,5]
        cell['reads_total'] = cell['reads_WT']+cell['reads_mutant']
        if cell['reads_total'] < MIN_READS:
            cell['CRISPResso_pass'] = False
            cell['well'] = file[file.find('CRISPResso_on_') + len('CRISPResso_on_'):file.find('CRISPResso_on_') + len('CRISPResso_on_')+4]
            cell['plate'] = file[file.find('_001_') + len('_001_'):file.find('_001_') + len('_001_')+3]
            cell['indel_WT'] = 'NA'
            cell['indel_mutant'] = 'NA'
        else:
            cell['CRISPResso_pass'] = True
            cell['well'] = file[file.find('CRISPResso_on_') + len('CRISPResso_on_'):file.find('CRISPResso_on_') + len('CRISPResso_on_')+4]
            cell['plate'] = file[file.find('_001_') + len('_001_'):file.find('_001_') + len('_001_')+3]
            cell['indel_WT'] = df_percentage.iloc[0,9]+df_percentage.iloc[0,10]
            cell['indel_mutant'] = df_percentage.iloc[1,9]+df_percentage.iloc[1,10]
    else:
        cell['well'] = file[file.find('CRISPResso_on_') + len('CRISPResso_on_'):file.find('CRISPResso_on_') + len('CRISPResso_on_')+4]
        cell['plate'] = file[file.find('_001_') + len('_001_'):file.find('_001_') + len('_001_')+3]
        cell['CRISPResso_pass'] = False
        cell['reads_total'] = 0
        cell['reads_WT'] = 0
        cell['reads_mutant'] = 0
        cell['indel_WT'] = 'NA'
        cell['indel_mutant'] = 'NA'
    data.append(cell)


 

In [ ]:
for cell in data: 
    cell['percent_WT'] = cell['reads_WT']/(cell['reads_WT']+cell['reads_mutant'])
    cell['percent_mutant'] = cell['reads_mutant']/(cell['reads_WT']+cell['reads_mutant'])
    cell['percent_indels_WT'] = cell['indel_WT']/cell['reads_WT']
    cell['percent_indels_mutant'] = cell['indel_mutant']/cell['reads_mutant']
    if cell['CRISPResso_pass']:
        if cell['percent_mutant'] < CO_MUT:
            cell['genotype'] = 'WT-WT'
        elif cell['percent_mutant'] < 1-CO_MUT:
            cell['genotype'] = 'WT-MUT'
        else:
            cell['genotype'] = 'MUT-MUT'
    else:
        cell['genotype'] = 'FAILED'




In [ ]:
for cell in data:
    if cell['CRISPResso_pass']:
        if cell['genotype'] == 'WT-WT':
            if cell['percent_indels_WT'] < CO_HOM:
                cell['genotype_edited'] = 'WT-WT'
            elif cell['percent_indels_WT'] < 1-CO_HOM:
                cell['genotype_edited'] = 'WT-WTx'
            else:
                cell['genotype_edited'] = 'WTx-WTx'
        elif cell['genotype'] == 'MUT-MUT':
            if cell['percent_indels_mutant'] < CO_HOM:
                cell['genotype_edited'] = 'MUT-MUT'
            elif cell['percent_indels_mutant'] < 1-CO_HOM:
                cell['genotype_edited'] = 'MUT-MUTx'
            else:
                cell['genotype_edited'] = 'MUTx-MUTx'
        elif cell['genotype'] == 'WT-MUT':
            if cell['percent_indels_WT'] < CO_HET and cell['percent_indels_mutant'] < 1-CO_HET:
                cell['genotype_edited'] = 'WT-MUT'
            elif cell['percent_indels_WT'] < CO_HET and cell['percent_indels_mutant'] < CO_HET:
                cell['genotype_edited'] = 'WT-MUTa' #%Indels on the mutant allele should be 0 or 1, so anything that isnt close to 0 or 1 is ambiguous
            elif cell['percent_indels_WT'] < CO_HET and cell['percent_indels_mutant'] > CO_HET:
                cell['genotype_edited'] = 'WT-MUTx'
            elif cell['percent_indels_WT'] > CO_HET and cell['percent_indels_mutant'] < CO_HET:
                cell['genotype_edited'] = 'WTx-MUT'
            elif cell['percent_indels_WT'] > CO_HET and cell['percent_indels_mutant'] > CO_HET:
                cell['genotype_edited'] = 'WTx-MUTx'
    else:
        cell['genotype_edited'] = 'FAILED'

df = pd.DataFrame(data)
df.to_csv('data_JAK2.csv', index=False)